In [ ]:
#Installing PyKeen
! pip install --upgrade pip
! python -c "import pykeen" || pip install git+https://github.com/pykeen/pykeen.git

In [ ]:
import torch
import pykeen
import pandas as pd
from pykeen import predict
from pykeen.datasets import Nations
from pykeen.pipeline import pipeline
from pykeen.triples import TriplesFactory
import numpy as np
from scipy.spatial.distance import euclidean
import itertools
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
file_path = 'query-result.tsv'
tf = TriplesFactory.from_path(file_path)
training, testing = tf.split([0.9,0.1])
#Using 90-10 split for training and testing because our dataset gives errors with 80-20 split

In [ ]:
def train_and_evaluate(model_name, embedding_dim, num_epochs, lr, num_negs_per_pos):
    result = pipeline(
        training=training,
        testing=testing,
        model=model_name,
        model_kwargs=dict(
            embedding_dim=embedding_dim,
        ),
        training_kwargs=dict(
            num_epochs=num_epochs
        ),
        optimizer_kwargs=dict(
            lr=lr,
        ),
        negative_sampler_kwargs=dict(
            num_negs_per_pos=num_negs_per_pos,
        ),
        random_seed=2025,
        device=device
    )
    return result

In [ ]:
models = ['TransE', 'TransH', 'ComplEx', 'RotatE']
embedding_dims = [128]
num_epochs_list = [20]
learning_rates = [0.01]
neg_samples = [1]

#This could be used as a gridline to find the best hyperparameters. 
#In this case we have only one hyperparameter for each category since training time is a constraint. 
#We needed to store and save every model in order to use them later for evaluation, so we are not using grid search here.

results = {}

In [ ]:
for model, dim, epochs, lr, negs in itertools.product(models, embedding_dims, num_epochs_list, learning_rates, neg_samples):
    print(f"Training {model} with dim={dim}, epochs={epochs}, lr={lr}, negs={negs}")
    result = train_and_evaluate(model, dim, epochs, lr, negs)
    key = f"{model}_dim{dim}_epochs{epochs}_lr{lr}_negs{negs}"
    results[key] = result.metric_results.to_dict()

In [ ]:
# Save results
import json
with open('kge_results.json', 'w') as f:
    json.dump(results, f)

best_model = max(results, key=lambda k: results[k]['both']['realistic']['hits_at_10'])
print(f"Best model: {best_model}")
print(f"Hits@10: {results[best_model]['both']['realistic']['hits_at_10']}")

best_model_name = best_model.split('_')[0]
best_dim = int(best_model.split('_')[1].replace('dim', ''))
best_epochs = int(best_model.split('_')[2].replace('epochs', ''))
best_lr = float(best_model.split('_')[3].replace('lr', ''))
best_negs = int(best_model.split('_')[4].replace('negs', ''))

final_model = train_and_evaluate(best_model_name, best_dim, best_epochs, best_lr, best_negs)
final_model.save_to_directory(f"best_{best_model_name}_model")